In [1]:
!pip install trino

In [ ]:
import trino
import pandas as pd
import numpy as np

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [8]:
conn = trino.dbapi.connect(host='192.168.1.71',port=8083,user='trino',catalog='hive', schema="data") 
cur = conn.cursor() 
cur.execute('show tables') 
i= cur.fetchall() 
i 

[['bankchurners'],
 ['bankloan'],
 ['diabetes'],
 ['german_credit_data'],
 ['gld_price_data'],
 ['heart'],
 ['insurance'],
 ['mobile'],
 ['predictive'],
 ['salary'],
 ['shipping'],
 ['shop_customer'],
 ['tesla'],
 ['walmart_sales']]

In [6]:
conn = trino.dbapi.connect(host='192.168.1.71',port=8083,user='trino',catalog='hive', schema="data") 
cur = conn.cursor() 
cur.execute('select * from tesla') 
f= cur.fetchall() 
f 

TrinoExternalError: TrinoExternalError(type=EXTERNAL, name=HIVE_METASTORE_ERROR, message="Failed connecting to Hive metastore: [192.168.1.71:9083]", query_id=20230403_064001_00006_b5b5p)

In [7]:
df=pd.DataFrame(f,columns=[i[0] for i in cur.description])

In [8]:
df

,date,open,high,low,close,adj close,volume
0,2010-06-29,19.000000,25.000000,17.540001,23.889999,23.889999,18766300
1,2010-06-30,25.790001,30.420000,23.299999,23.830000,23.830000,17187100
2,2010-07-01,25.000000,25.920000,20.270000,21.959999,21.959999,8218800
3,2010-07-02,23.000000,23.100000,18.709999,19.200001,19.200001,5139800
4,2010-07-06,20.000000,20.000000,15.830000,16.110001,16.110001,6866900
...,...,...,...,...,...,...,...
460,2012-04-25,32.070000,32.990002,32.070000,32.910000,32.910000,712200
461,2012-04-26,32.959999,33.520000,32.910000,33.490002,33.490002,425300
462,2012-04-27,33.599998,33.630001,32.910000,33.340000,33.340000,591000
463,2012-04-30,33.270000,33.360001,32.580002,33.130001,33.130001,413900


In [9]:
x=df.iloc[:,1:6]

In [11]:
x

,open,high,low,close,adj close
0,19.000000,25.000000,17.540001,23.889999,23.889999
1,25.790001,30.420000,23.299999,23.830000,23.830000
2,25.000000,25.920000,20.270000,21.959999,21.959999
3,23.000000,23.100000,18.709999,19.200001,19.200001
4,20.000000,20.000000,15.830000,16.110001,16.110001
...,...,...,...,...,...
460,32.070000,32.990002,32.070000,32.910000,32.910000
461,32.959999,33.520000,32.910000,33.490002,33.490002
462,33.599998,33.630001,32.910000,33.340000,33.340000
463,33.270000,33.360001,32.580002,33.130001,33.130001


In [12]:
y=df.iloc[:,-1]
y

0      18766300
1      17187100
2       8218800
3       5139800
4       6866900
         ...   
460      712200
461      425300
462      591000
463      413900
464      659000
Name: volume, Length: 465, dtype: int64

In [13]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=30)

In [14]:
models = [
    ('Linear Regression', LinearRegression()),
    ('Decision Tree Regressor', DecisionTreeRegressor()),
    ('Random Forest Regressor', RandomForestRegressor()),
    ('Support Vector Regression', SVR(kernel='linear'))
]

In [15]:
for name, model in models:
    # Fit model
    model.fit(xtrain,ytrain)
    # Make predictions
    ypred = model.predict(xtest)
    

In [16]:
# Calculate mean squared error
mse = mean_squared_error(ytest,ypred)
    # Print results    
print(f'{name} - MSE: {mse:.2f}')


Support Vector Regression - MSE: 4479854166890.60


In [17]:
!pip install mlflow

In [18]:
import mlflow


In [19]:
!pip install pymysql

In [20]:
import pymysql

In [21]:
#mlflow.start_run()

In [23]:
import os 
from random import random, randint 
from mlflow import log_metric, log_param, log_artifacts 
 
if __name__ == "__main__": 
    # Log a parameter (key-value pair)
    mlflow.set_experiment("my_existing_experiment")
    log_param("param2", randint(0, 100)) 
 
    # Log a metric; metrics can be updated throughout the run 
    log_metric("foo", random()) 
    log_metric("foo", random () + 1) 
    log_metric("foo", random () + 2) 
 
    # Log an artifact (output file) 
    if not os.path.exists("outputs"): 
        os.makedirs("outputs") 
    with open ("outputs/test.txt", "w") as f: 
        f.write("hello world!") 
    log_artifacts("outputs")

MlflowException: Changing param values is not allowed. Param with key='param2' was already logged with value='3' for run ID='15df9095198b40c89c7f6d04eae4de3e'. Attempted logging new value '25'.

The cause of this error is typically due to repeated calls
to an individual run_id event logging.

Incorrect Example:
---------------------------------------
with mlflow.start_run():
    mlflow.log_param("depth", 3)
    mlflow.log_param("depth", 5)
---------------------------------------

Which will throw an MlflowException for overwriting a
logged parameter.

Correct Example:
---------------------------------------
with mlflow.start_run():
    with mlflow.start_run(nested=True):
        mlflow.log_param("depth", 3)
    with mlflow.start_run(nested=True):
        mlflow.log_param("depth", 5)
---------------------------------------

Which will create a new nested run for each individual
model and prevent parameter key collisions within the
tracking store.'

In [24]:
#mlflow.end_run()

In [25]:
mlflow.get_experiment_by_name("my_existing_experiment")

<Experiment: artifact_location='file:///root/zigma/mlruns/344423630892432606', creation_time=1680156290447, experiment_id='344423630892432606', last_update_time=1680156290447, lifecycle_stage='active', name='my_existing_experiment', tags={}>

In [26]:
# Check if there is an active run
if mlflow.active_run() is not None:
    # End the active run
    mlflow.end_run()

# Start an MLflow run context
with mlflow.start_run():
    # Get the currently active run
    active_run = mlflow.active_run()

    # Check if there is an active run
    if active_run is not None:
        print("Active run ID:", active_run.info.run_id)
        print("Active experiment ID:", active_run.info.experiment_id)
    else:
        print("There is no active run.")


Active run ID: 4e4a3d3db4324cf09c73c70953c4db09
Active experiment ID: 344423630892432606


In [27]:
mlflow.store.tracking.file_store

<module 'mlflow.store.tracking.file_store' from '/usr/local/lib/python3.9/site-packages/mlflow/store/tracking/file_store.py'>

In [29]:
mlflow.get_tracking_uri()

'file:///root/zigma/mlruns'

In [1]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 1.7 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 9.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824027 sha256=19731c3d83cca46020dc22dffbfab88026da969be830bf003c26726957d8b744
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [5]:
df=SparkSession.builder.appName("hellow world").getOrCreate()

JAVA_HOME is not set


RuntimeError: Java gateway process exited before sending its port number

In [6]:
hello_df = spark.createDataFrame([("Hello, World!",)], ["message"])

# Show the DataFrame
hello_df.show()

# Stop the SparkSession
spark.stop()

NameError: name 'spark' is not defined